In [31]:
from nltk.tokenize import RegexpTokenizer
import cPickle as pickle
import string
import os
import re

%matplotlib inline
import numpy as np
import random
import time

import logging
from logging import info

from multiprocessing import Pool as ThreadPool
import itertools

import nltk

from thesis.utils.text import get_sentences, sentence_wordtokenizer

In [2]:
root = logging.getLogger()
for handler in root.handlers[:]:
    root.removeHandler(handler)
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO) # adds a default StreamHanlder
#root.addHandler(logging.StreamHandler())

In [3]:
SAMPLE_RATIO = 0.01

In [4]:
root_location = "/big/s/shalaby/"
exports_location = root_location + "exported_data/"

# training_file = root_location + 'docs_output_training_validation_documents_' + str(SAMPLE_RATIO)
training_file = root_location + 'docs_output.json'

doc_classifications_map_file = exports_location + "doc_classification_map.pkl"
sections_file = exports_location + "sections.pkl"
classes_file = exports_location + "classes.pkl"
subclasses_file = exports_location + "subclasses.pkl"
classifications_output = exports_location + "classifications.pkl"
# training_docs_list_file = exports_location + "training_documents_" + str(SAMPLE_RATIO) + "_sample.pkl"
# validation_docs_list_file = exports_location + "validation_documents_" + str(SAMPLE_RATIO) + "_sample.pkl"
training_docs_list_file = exports_location + "extended_pv_training_docs_list.pkl"
validation_docs_list_file = exports_location + "extended_pv_validation_docs_list.pkl"
test_docs_list_file = exports_location + "extended_pv_test_docs_list.pkl"

In [5]:
%%time
doc_classification_map = pickle.load(open(doc_classifications_map_file))
sections = pickle.load(open(sections_file))
classes = pickle.load(open(classes_file))
subclasses = pickle.load(open(subclasses_file))
training_docs_list = pickle.load(open(training_docs_list_file))
validation_docs_list = pickle.load(open(validation_docs_list_file))
test_docs_list = pickle.load(open(test_docs_list_file))

CPU times: user 28.3 s, sys: 4.58 s, total: 32.9 s
Wall time: 32.9 s


In [6]:
len(training_docs_list)

120156

In [7]:
len(validation_docs_list)

29675

In [8]:
len(test_docs_list)

37771

In [9]:
MIN_SIZE = 0
NUMBER_INDICATOR = "number_inidicator"
CURRENCY_INDICATOR = "currency_inidicator"
CHEMICAL_INDICATOR = "chemical_inidicator"

In [21]:
string.punctuation

'!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~'

In [28]:
re.findall(r"[\w']+|[.,!?;]", ',text".?')

[',', 'text', '.', '?']

In [56]:
re.findall(r"^([,!?]*)(.+?)([,!?]?)$", "U.S.")[0]

('', 'U.S.', '')

In [68]:
punctuation_to_strip = '"#%&\'();:*+-/<=>@[\\]^_`{|}~'
def stemtokenizer(text):
    """ MAIN FUNCTION to get clean stems out of a text. A list of clean stems are returned """
    tokenizer = RegexpTokenizer(r'\s+', gaps=True)
    tokens = tokenizer.tokenize(text)
    stems = []  # result
    for token in tokens:
        stem = token.lower()
        stem = stem.strip(punctuation_to_strip)
        if stem:
            if is_number(stem):
                token_stems = [NUMBER_INDICATOR]
            elif is_currency(stem):
                token_stems = [CURRENCY_INDICATOR]
            elif is_chemical(stem):
                token_stems = [CHEMICAL_INDICATOR]
            else:
                stem = stem.strip(punctuation_to_strip)
                token_stems = list(re.findall(r"^([.,!?;:]*)(.+?)([.,!?;:]?)$", stem)[0])
                if len(token_stems):
                    if token_stems[1] in abbrev_types_set:
                        token_stems[1] = token_stems[1] + token_stems[2]
                        token_stems[2] = ""
                
            for stem in token_stems:   
                if stem and len(stem) >= MIN_SIZE:
                    # extract uni-grams
                    stems.append(stem)
    del tokens
    return stems

def is_number(str):
    """ Returns true if given string is a number (float or int)"""
    try:
        float(str.replace(",", ""))
        return True
    except ValueError:
        return False

def is_currency(str):
    return str[0] == "$"

def is_chemical(strg):
    return (strg.count("-") > 2 and len(strg) >= 25) or len(strg) >= 40

MIN_SENT_LENGTH = 50
MAX_NON_TABLE_ROW_LENGTH = 100

def is_table_row(sent):
    return len(sent) < MAX_NON_TABLE_ROW_LENGTH and sent.count('.') > 2

def min_sentence_length_enforcer(sentences):
    # initialize with '' so we dont have to add a condition to be evaluated in the loop
    new_sentences = ['']
    for sent in sentences:
        if len(sent) < MIN_SENT_LENGTH or is_table_row(sent):
            new_sentences[-1] += ' ' + sent
        else:
            new_sentences.append(sent)
    if len(new_sentences[0]) == 0: new_sentences = new_sentences[1:]
    return new_sentences

def get_sentences(text):
    sents = sentence_tokenizer.tokenize(text)
    sents = min_sentence_length_enforcer(sents)
    return sents

reload(nltk.data)
sentence_tokenizer = nltk.data.load('tokenizers/punkt/english.pickle')
extra_abbrv = [u'u.s', u'fig', u'figs', u'no', u'ser',
               u'jan', u'feb', u'mar', u'apr', u'may', u'jun', u'jul', u'aug', u'sep', u'oct', u'nov', u'dec',
u'proc', u'natl', u'sci', u'al', u'biochem', u'mol', u'res', u'biophys', u'commun', u'acad', 
u'chem', u'med', u'biol', u'enzymol', u'Am', u'Soc', u'pat', u'nos', u'id', u'seq', 
u'gen', u'ed', u'publ', u'cell', u'ii', u'iii', u'iv', u'viral', u'dis', u'infect', 
u'rev', u'supp', u'dev', u'pp', u'genet', u'pp', u'nucl', u'pub', u'etc', u'virol', 
u'u.s. pat', u'u.s. ser', u'u.s. patent', u'ann', u'microbiol', u'environ', u'U.S',
u'curr', u'vol', u'enz', u'struct', u'exp', u'approx', u'int', u'oncol', u'appl', 
u'math', u'adv', u'u.s.c', u'et', u'app', u'biosci', u'molec']
extra_abbrv.extend([str(i) for i in range(1,40)])
extra_abbrv.extend(list('abcdefghijklmnopqrstuvwxyz'))
sentence_tokenizer._params.abbrev_types.update(extra_abbrv)
abbrev_types_set = set(sentence_tokenizer._params.abbrev_types)

In [58]:
dd = "CROSS REFERENCE TO RELATED APPLICATIONS This application is a Divisional of U.S. application Ser. No. 09/466,778, filed on Dec. 20, 1999, now U.S. Pat. No. 6,872,546, which claims benefit under 35 U.S.C. \xa7 119(e) of U.S."

In [59]:
dd = "The invention encompasses full-length WF-HABP polynucleotides; host cell expression systems; encompasses full-length WF-HABP polypeptides (including fragments, variants, derivatives and analogs thereof); encompasses full-length WF-HABP fusion proteins; antibodies encompasses full-length WF-HABP; agonists and antagonists encompasses full-length WF-HABP; and other compounds that modulate encompasses full-length WF-HABP gene expression or encompasses full-length WF-HABP activity; that can be used for diagnosis, drug screening, and treatment or prevention of disorders which include, but are not limited to, vascular disorders and conditions, congenital pain insensitivity, inflammation, ischemia, host defense dysfunction, immune surveillance dysfunction, neural disorders, arthritis, edema, multiple sclerosis, autoimmunity, immune dysfunction, cancers, metastasis, integumentary disorders, and allergy."

In [60]:
stemtokenizer(dd)

['the',
 'invention',
 'encompasses',
 'full-length',
 'wf-habp',
 'polynucleotides',
 'host',
 'cell',
 'expression',
 'systems',
 'encompasses',
 'full-length',
 'wf-habp',
 'polypeptides',
 'including',
 'fragments',
 ',',
 'variants',
 ',',
 'derivatives',
 'and',
 'analogs',
 'thereof',
 'encompasses',
 'full-length',
 'wf-habp',
 'fusion',
 'proteins',
 'antibodies',
 'encompasses',
 'full-length',
 'wf-habp',
 'agonists',
 'and',
 'antagonists',
 'encompasses',
 'full-length',
 'wf-habp',
 'and',
 'other',
 'compounds',
 'that',
 'modulate',
 'encompasses',
 'full-length',
 'wf-habp',
 'gene',
 'expression',
 'or',
 'encompasses',
 'full-length',
 'wf-habp',
 'activity',
 'that',
 'can',
 'be',
 'used',
 'for',
 'diagnosis',
 ',',
 'drug',
 'screening',
 ',',
 'and',
 'treatment',
 'or',
 'prevention',
 'of',
 'disorders',
 'which',
 'include',
 ',',
 'but',
 'are',
 'not',
 'limited',
 'to',
 ',',
 'vascular',
 'disorders',
 'and',
 'conditions',
 ',',
 'congenital',
 'pain',
 

## Get Separate Training and Validation Data Files

In [10]:
BATCH_SIZE = 10000

preprocessed_location = root_location + "preprocessed_data/"
TRAINING_PREPROCESSED_FILES_PREFIX = preprocessed_location + "extended_pv_training_docs_data_preprocessed-"
VALIDATION_PREPROCESSED_FILES_PREFIX = preprocessed_location + "extended_pv_validation_docs_data_preprocessed-"
TEST_PREPROCESSED_FILES_PREFIX = preprocessed_location + "extended_pv_test_docs_data_preprocessed-"

In [11]:
SENTENCE_ID = "{}_s{}"

In [12]:
def write_batch(file_prefix, batch_lines, batch_start):
    if len(batch_lines):
        print "writing batch %d" % batch_start
        with open(file_prefix + str(batch_start), 'w') as batch_file:
            for line in batch_lines:
                batch_file.write((u" ".join(line) + "\n").encode('utf-8'))

## Line Positions

Load Line Positions

In [13]:
line_positions = pickle.load(open("/big/s/shalaby/exported_data/line_positions.pkl", "r"))

Compute Line Positions

In [9]:
%%time
line_positions = dict()
with open(training_file) as f:
    
    i = 0
    line_positions[i] = f.tell()
    line = f.readline()
    while line:
        i+=1
        if not line.strip(): continue
        line_positions[i] = f.tell()
        line = f.readline()
    del line_positions[i]

CPU times: user 3min 54s, sys: 30.6 s, total: 4min 25s
Wall time: 4min 53s


In [12]:
pickle.dump(line_positions, open("/big/s/shalaby/exported_data/line_positions.pkl", "w"))

In [14]:
len(line_positions)

2009750

## Doc Line Positions

Load Doc Line Positions

In [14]:
doc_line_positions = pickle.load(open(exports_location + "doc_line_positions.pkl", "r"))

In [15]:
len(doc_line_positions)

2009750

Compute Doc Line Positions

In [30]:
%%time
doc_line_positions = dict()
with open(training_file) as f:

    i = 0
    curr_pos = f.tell()
    line = f.readline()
    while line:
        doc_id, text = eval(line)
        if not line.strip(): continue
        doc_line_positions[doc_id] = curr_pos
        curr_pos = f.tell()
        line = f.readline()
        i+=1
        if i % 1000:
            info("Finished: {}".format(i))

CPU times: user 19min 7s, sys: 35.7 s, total: 19min 43s
Wall time: 19min 47s


In [32]:
pickle.dump(doc_line_positions, open(exports_location + "doc_line_positions.pkl", "w"))

# Training


In [28]:
DOCS_LIST = training_docs_list
FILE_PREFIX = TRAINING_PREPROCESSED_FILES_PREFIX

In [32]:
def multithreaded_batch_creation(start_index):

    if os.path.exists(FILE_PREFIX + str(start_index)):
        info("Batch {} already exists, skipping..".format(start_index))
        return
    
    info("Batch creation working on {}\n".format(start_index))
    with open(training_file) as file_obj:
        token_lines = []
        start_time = time.time()
        for doc_index, curr_doc_id in enumerate(DOCS_LIST[start_index:]):
            file_obj.seek(doc_line_positions[curr_doc_id])
            line = file_obj.readline()
            (doc_id, text) = eval(line)

            sent_tokens_list = []
            doc_tokens_list = [doc_id]

            # get the tokenized sentences
            sentences = get_sentences(text)
            for i,sent in enumerate(sentences):
                sentence_id = SENTENCE_ID.format(doc_id, i+1)
                sent_tokens_list.append([sentence_id] + sentence_wordtokenizer(sent))

            # create the document tokens list from the sentence token lists instead of tokenizing the whole document again
            for sent_tokens in sent_tokens_list:
                doc_tokens_list.extend(sent_tokens[1:])

            # now add the document tokens and the sentence tokens to the list that will be written to the file
            token_lines.append(doc_tokens_list)
            token_lines.extend(sent_tokens_list)

            if doc_index % 1000 == 0: info("Doc:{:6} -> Total Lines to write: {:8}".format(start_index + doc_index, len(token_lines)))
            if doc_index >= BATCH_SIZE - 1:
                break
    duration = time.time() - start_time
    info("Finished batch {} of size {:d} in {:.0f}m {:.0f}s".format(start_index, BATCH_SIZE, * divmod(duration, 60)))
    info("For index {}, the actual number of lines written is: {}".format(start_index, len(token_lines)))
    
    write_batch(FILE_PREFIX, token_lines, start_index)
    del token_lines

In [18]:
SAMPLE_SIZE = len(training_docs_list)

In [19]:
(divmod(SAMPLE_SIZE, BATCH_SIZE)[0]+1) * BATCH_SIZE

130000

In [20]:
batches = range(0, (divmod(SAMPLE_SIZE, BATCH_SIZE)[0]+1) * BATCH_SIZE, BATCH_SIZE )

In [ ]:
%%time
try:
    pool = ThreadPool(16)
    # +1 since range is end-exclusive
    batches = range(0, (divmod(SAMPLE_SIZE, BATCH_SIZE)[0]+1) * BATCH_SIZE, BATCH_SIZE )
    indices = pool.map(multithreaded_batch_creation, batches)
    pool.close()
    pool.terminate()
finally:
    pool.close()
    pool.terminate()

2017-03-05 03:15:59,531 : INFO : Batch creation working on 40000

2017-03-05 03:15:59,531 : INFO : Batch creation working on 20000

2017-03-05 03:15:59,531 : INFO : Batch creation working on 30000

2017-03-05 03:15:59,531 : INFO : Batch creation working on 0

2017-03-05 03:15:59,531 : INFO : Batch creation working on 10000

2017-03-05 03:15:59,533 : INFO : Batch creation working on 90000

2017-03-05 03:15:59,532 : INFO : Batch creation working on 60000

2017-03-05 03:15:59,533 : INFO : Batch creation working on 70000

2017-03-05 03:15:59,532 : INFO : Batch creation working on 50000

2017-03-05 03:15:59,533 : INFO : Batch creation working on 80000

2017-03-05 03:15:59,534 : INFO : Batch creation working on 100000

2017-03-05 03:15:59,536 : INFO : Batch creation working on 110000

2017-03-05 03:15:59,536 : INFO : Batch creation working on 120000

2017-03-05 03:15:59,613 : INFO : 58
2017-03-05 03:15:59,656 : INFO : 103
2017-03-05 03:15:59,656 : INFO : 127
2017-03-05 03:15:59,663 : INFO : 

writing batch 120000


2017-03-05 03:19:15,160 : INFO : 203424
2017-03-05 03:19:18,040 : INFO : 228307
2017-03-05 03:19:24,655 : INFO : 241057
2017-03-05 03:19:29,227 : INFO : 229779
2017-03-05 03:19:33,072 : INFO : 242969
2017-03-05 03:19:33,568 : INFO : 229559
2017-03-05 03:19:38,542 : INFO : 233298
2017-03-05 03:19:39,541 : INFO : 234948
2017-03-05 03:19:42,106 : INFO : 254152
2017-03-05 03:19:43,240 : INFO : 244518
2017-03-05 03:19:44,998 : INFO : 256308
2017-03-05 03:20:04,229 : INFO : 279122
2017-03-05 03:22:19,521 : INFO : 400542
2017-03-05 03:22:49,276 : INFO : 461417
2017-03-05 03:23:03,220 : INFO : 476057
2017-03-05 03:23:03,576 : INFO : 447324
2017-03-05 03:23:05,930 : INFO : 454121
2017-03-05 03:23:08,328 : INFO : 484385
2017-03-05 03:23:10,075 : INFO : 457522
2017-03-05 03:23:15,167 : INFO : 497750
2017-03-05 03:23:24,213 : INFO : 505281
2017-03-05 03:23:26,933 : INFO : 485703
2017-03-05 03:23:38,007 : INFO : 497345
2017-03-05 03:24:02,592 : INFO : 547861
2017-03-05 03:25:46,599 : INFO : 605961


# Validation

In [75]:
DOCS_LIST = validation_docs_list
FILE_PREFIX = VALIDATION_PREPROCESSED_FILES_PREFIX

In [76]:
SAMPLE_SIZE = len(validation_docs_list)

In [78]:
%%time
try:
    pool = ThreadPool(16)
    # +1 since range is end-exclusive
    batches = range(0, (divmod(SAMPLE_SIZE, BATCH_SIZE)[0]+1) * BATCH_SIZE, BATCH_SIZE )
    indices = pool.map(multithreaded_batch_creation, batches)
    pool.close()
    pool.terminate()
except e as Exception:
    pool.close()
    pool.terminate()
    raise e

2017-03-04 18:02:17,370 : INFO : Batch creation working on 20000

2017-03-04 18:02:17,370 : INFO : Batch creation working on 10000

2017-03-04 18:02:17,369 : INFO : Batch creation working on 0

2017-03-04 18:02:17,662 : INFO : 56
2017-03-04 18:02:17,852 : INFO : 65
2017-03-04 18:02:17,978 : INFO : 309
2017-03-04 18:05:18,878 : INFO : 216333
2017-03-04 18:05:40,903 : INFO : 245047
2017-03-04 18:05:42,567 : INFO : 257709
2017-03-04 18:08:13,537 : INFO : 427212
2017-03-04 18:08:56,333 : INFO : 495800
2017-03-04 18:08:59,727 : INFO : 507128
2017-03-04 18:11:19,354 : INFO : 639322
2017-03-04 18:12:06,057 : INFO : 732933
2017-03-04 18:12:21,122 : INFO : 758304
2017-03-04 18:14:23,184 : INFO : 846145
2017-03-04 18:15:06,734 : INFO : 958711
2017-03-04 18:15:45,331 : INFO : 1014866
2017-03-04 18:17:29,865 : INFO : 1061395
2017-03-04 18:18:10,942 : INFO : 1195757
2017-03-04 18:19:11,428 : INFO : 1277012
2017-03-04 18:20:39,682 : INFO : 1265578
2017-03-04 18:21:37,980 : INFO : 1417192
2017-03-04 

writing batch 0


2017-03-04 18:35:25,363 : INFO : Finished batch of 10000 in 33m 8s
2017-03-04 18:35:25,365 : INFO : For index 10000, the actual number of lines written is: 2349706


writing batch 10000


2017-03-04 18:36:30,630 : INFO : Finished batch of 10000 in 34m 13s
2017-03-04 18:36:30,632 : INFO : For index 20000, the actual number of lines written is: 2438579


writing batch 20000
CPU times: user 1.66 s, sys: 1.49 s, total: 3.15 s
Wall time: 34min 49s


# Test Data

In [ ]:
DOCS_LIST = test_docs_list
FILE_PREFIX = TEST_PREPROCESSED_FILES_PREFIX

In [ ]:
SAMPLE_SIZE = len(test_docs_list)

In [ ]:
SAMPLE_SIZE

In [ ]:
%%time
try:
    pool = ThreadPool(16)
    # +1 since range is end-exclusive
    batches = range(0, (divmod(SAMPLE_SIZE, BATCH_SIZE)[0]+1) * BATCH_SIZE, BATCH_SIZE )
    indices = pool.map(multithreaded_batch_creation, batches)
    pool.close()
    pool.terminate()
except e as Exception:
    pool.close()
    pool.terminate()
    raise e

In [22]:
pool.close()
pool.terminate()